In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np

In [3]:
#========
# Setup
#========
events = pd.DataFrame({
    "ts": pd.to_datetime([
        "2025-12-01 09:10" , "2025-12-01 16:30" , "2025-12-02 11:05" ,
        "2025-12-04 14:20" , "2025-12-04 18:45" , "2025-12-07 10:15"
    ]) ,
    "sales": [40 , 80 , 55 , 60 , 50 , 90] ,
    "returns": [1 , 2 , 1 , 0 , 1 , 2] ,
})
ts = events.sort_values("ts").set_index("ts")
ts

,sales,returns
ts,,
2025-12-01 09:10:00,40,1
2025-12-01 16:30:00,80,2
2025-12-02 11:05:00,55,1
2025-12-04 14:20:00,60,0
2025-12-04 18:45:00,50,1
2025-12-07 10:15:00,90,2


In [4]:
#=======================================
# Case 1) Resample into daily buckets
#=======================================
daily = ts.resample("D").agg({"sales": "sum" , "returns": "sum"})
daily

,sales,returns
ts,,
2025-12-01,120,3
2025-12-02,55,1
2025-12-03,0,0
2025-12-04,110,1
2025-12-05,0,0
2025-12-06,0,0
2025-12-07,90,2


In [5]:
#======================================
# Case 2) Control bucket boundaries
#======================================
weekly = ts.resample("W-SUN" , label = "right" , closed = "right").agg({"sales": "sum" , "returns": "sum"})
weekly

,sales,returns
ts,,
2025-12-07,375,7


In [6]:
#=======================================================
# Case 3) Enforce daily frequency without aggregation
#=======================================================
daily_sparse = daily.drop(daily.index[[1 , 3]])
daily_asfreq = daily_sparse.asfreq("D")
daily_asfreq

,sales,returns
ts,,
2025-12-01,120.0,3.0
2025-12-02,NaN,NaN
2025-12-03,0.0,0.0
2025-12-04,NaN,NaN
2025-12-05,0.0,0.0
2025-12-06,0.0,0.0
2025-12-07,90.0,2.0


In [7]:
#=====================================
# Case 4) Fill gaps intentionally
#=====================================
filled_ffill = daily_asfreq.ffill()
filled_ffill

filled_interp = daily_asfreq.interpolate()
filled_interp.round(2)

,sales,returns
ts,,
2025-12-01,120.0,3.0
2025-12-02,120.0,3.0
2025-12-03,0.0,0.0
2025-12-04,0.0,0.0
2025-12-05,0.0,0.0
2025-12-06,0.0,0.0
2025-12-07,90.0,2.0


,sales,returns
ts,,
2025-12-01,120.0,3.0
2025-12-02,60.0,1.5
2025-12-03,0.0,0.0
2025-12-04,0.0,0.0
2025-12-05,0.0,0.0
2025-12-06,0.0,0.0
2025-12-07,90.0,2.0


In [8]:
#===============================================
# Case 5) PeriodIndex for reporting periods
#===============================================
monthly_period = daily.copy()
monthly_period.index = monthly_period.index.to_period("M")
monthly_report = monthly_period.groupby(level = 0).sum()
monthly_report

,sales,returns
ts,,
2025-12,375,7


In [9]:
#====================================================
# Case 6) Time-based rolling vs row-based rolling
#====================================================
daily_full = daily.asfreq("D")
daily_full["sales_roll_3rows"] = daily_full["sales"].rolling(window = 3 , min_periods = 1).mean()
daily_full["sales_roll_7days"] = daily_full["sales"].rolling(window = "7D" , min_periods = 1).mean()
daily_full.round(2)

,sales,returns,sales_roll_3rows,sales_roll_7days
ts,,,,
2025-12-01,120,3,120.00,120.00
2025-12-02,55,1,87.50,87.50
2025-12-03,0,0,58.33,58.33
2025-12-04,110,1,55.00,71.25
2025-12-05,0,0,36.67,57.00
2025-12-06,0,0,36.67,47.50
2025-12-07,90,2,30.00,53.57
